# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [1]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

from math import sqrt

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)
    
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [6]:
import numpy as np

In [24]:
def solve_tsp_nearest_neighbour(instance):
    n = len(instance)
    perm = [0]
    used = np.array([False] * n)
    used[0] = True
    last = 0
    
    for it in range(n - 1):
        best = -1
        for i in range(n):
            if (used[i] == True):
                continue
            
            if (best == -1 or (euclidean_distance(instance[last], instance[i]) < euclidean_distance(instance[last], instance[best]))):
                best = i
                
        used[best] = True
        perm = np.append(perm, [best])
        last = best
    return perm

In [49]:
def solve_tsp_nearest_insertion(instance):
    n = len(instance)
    perm = np.array([0])
    used = np.array([False] * n)
    used[0] = True
    
    for it in range(n - 1):
        best = -1
        best_dst = 1e18
        
        for i in range(n):
            if (used[i]):
                continue
            
            dst = 1e18
            
            for j in range(len(perm)):
                dst = min(dst, euclidean_distance(instance[i], instance[perm[j]]))
                
            if (dst < best_dst):
                best = i
                best_dst = dst
        
        if (len(perm) == 1):
            used[best] = True
            perm = np.append(perm, [best])
            continue
        
        pos = 0;
        best_dst = 1e18
        
        for i in range(len(perm)):
            now_dst = euclidean_distance(instance[perm[i]], instance[best])
            now_dst += euclidean_distance(instance[perm[(i + 1) % len(perm)]], instance[best])
            now_dst -= euclidean_distance(instance[perm[i]], instance[perm[(i + 1) % len(perm)]])
            
            if (now_dst < best_dst):
                best_dst = now_dst
                pos = i
        
        perm = np.append(perm, [best])
        posit = len(perm) - 1
        
        while (posit != pos + 1):
            perm[posit], perm[posit - 1] = perm[posit - 1], perm[posit]
            posit -= 1
        
        used[best] = True
    #print(perm)
    return perm

In [27]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour length {} using NI'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [50]:
run_all()

Solving instance d198.tsp… done in 0.22 seconds with tour length 18620 using NN and in 3.6 seconds with tour length 18052 using NI
Solving instance d493.tsp… done in 1.3 seconds with tour length 43646 using NN and in 5.9e+01 seconds with tour length 41576 using NI
Solving instance d657.tsp… done in 2.4 seconds with tour length 62176 using NN and in 1.4e+02 seconds with tour length 60195 using NI
Solving instance d2103.tsp… done in 2.3e+01 seconds with tour length 87468 using NN and in 4.7e+03 seconds with tour length 86027 using NI
Solving instance pr107.tsp… done in 0.064 seconds with tour length 46678 using NN and in 0.66 seconds with tour length 53211 using NI
Solving instance pr152.tsp… done in 0.13 seconds with tour length 85702 using NN and in 1.8 seconds with tour length 86914 using NI
Solving instance pr439.tsp… done in 1.1 seconds with tour length 131282 using NN and in 4.1e+01 seconds with tour length 132780 using NI


## Выводы
Как мы знаем второе решение хуже первого ассимптотически, но по качеству он выигрывает далеко не всегда, на файлах типа d, он выигрывает не сильно, а на файлах типа pr проигрывает, кроме pr152.